### Regression Analysis Project

#### Web Automation

In [ ]:
import requests
import urllib.request
import time

In [ ]:
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup

In [ ]:
url= 'link to site'
response=requests.get(url)

In [ ]:
response #200 confirms a response was obtained, that is it went through

In [ ]:
xpath_1 = '//*[@id="mainListingsWrapper"]/div/div/div[9]/div/div/ul/li[3]/a'
xpath_2 = '//*[@id="mainListingsWrapper"]/div/div/div[9]/div/div/ul/li[4]/a'
xpath_repeat = '//*[@id="mainListingsWrapper"]/div/div/div[9]/div/div/ul/li[5]/a'
xpaths = [xpath_1,xpath_2,xpath_repeat]
xpaths.extend([xpath_repeat for i in range(3600)])
# xpaths.append(xpath2)
# xpaths.append(xpath1)


In [ ]:
soup = BeautifulSoup(response.text, 'html.parser')

def crawl_auto(url,xpaths,cars,prices,mileage,location):

    driver = webdriver.Chrome("/Users/TaylorMK23/Downloads/chromedriver")
    driver.get(url)
    time.sleep(15)
    driver.refresh()
    for i,xpath in enumerate(xpaths):

        content = driver.page_source
        soup = BeautifulSoup(content)

        for data in soup.findAll('div', attrs={'class':'col-xs-12 result-item-inner'}):
            try:
                car_desc = data.find('span', attrs={'itemprop':'itemOffered'})
                cars.append(car_desc.text)
            except AttributeError:        
                car_name_nan_text = 'NaN'
                cars.append(car_name_nan_text)
            
            try:
                car_price = data.find('span', attrs={'class':'price-amount'})
                prices.append(car_price.text)
            except AttributeError:
                car_price_nan_text = 'NaN'
                prices.append(car_price_nan_text)
                
            try:
                miles = data.find('div', attrs={'class':'kms'})
                mileage.append(miles.text)
            except AttributeError:
                miles_nan_text = 'NaN'
                mileage.append(miles_nan_text)
            
            try:
                loc = data.find('span', attrs={'class':'proximity-text'})
                location.append(loc.text)
            except AttributeError:
                loc_nan_text = 'In Toronto, 15 km'
                location.append(loc_nan_text)
                
#         time.sleep(5)    
        WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, xpath))).click()
        time.sleep(5)
        url = driver.current_url
        driver.get(url)

In [ ]:
cars=[]
prices = []
mileages=[]
location=[]
crawl_auto(url,xpaths,cars,prices,mileages,location)

In [ ]:
S = {'Cars':cars,'Price':prices,'Mileage':mileages,'Location':location}

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
scraped = pd.DataFrame(S)

In [ ]:
scraped.to_csv('Scraped2.csv')

#### Data Cleaning

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
scraped = pd.read_csv('Scraped.csv')
scraped.head()

In [ ]:
cars_obj=scraped['Cars']
prices_obj=scraped['Price']
mileages_obj=scraped['Mileage']
location_obj=scraped['Location']

cars = [str(item) for item in cars_obj]
prices = [str(item) for item in prices_obj]
mileages = [str(item) for item in mileages_obj]
location = [str(item) for item in location_obj]

In [ ]:
year=[]
make=[]
model=[]
price=[]
mileage=[]
vendor_location=[]
dist_from_TO=[]
char1 = ' '
char2 = ', '
char3 = ' km'

for i in range(len(cars)):
    cars_list = cars[i].strip().split(' ')
    
    if len(cars_list) > 1:
        car_year = cars_list[0]
        car_make = cars_list[1]
        car_model = cars_list[2]
        year.append(int(car_year))
        make.append(car_make)
        model.append(car_model)
    elif cars_list[0] == 'NaN' or cars_list[0] == 'nan':
        year.append(np.NaN)
        make.append(np.NaN)
        model.append(np.NaN)
    else:
        car_year = cars_list[0]
        car_make = (np.NaN)
        car_model = (np.NaN)
        year.append(int(car_year))
        make.append(car_make)
        model.append(car_model)
    
    price_string = prices[i].replace('$','').replace(',','')
    if price_string == 'NaN' or price_string =='nan':
        price.append((np.NaN))
        
    else:
#         p = int(prices[i].replace('$','').replace(',',''))
        price.append(int(price_string))
        
    if type(mileages[i]) == float:
        
        m = mileages[i]
        m = 0
        mileage.append(m)
        
    else:
        if len(mileages[i].strip().split(' ')) > 1:
            m = int(mileages[i].strip().split(' ')[1].replace(',',''))
            mileage.append(m)

        else:
            m = (np.NaN)
            mileage.append(m)
    
#     l = location[i]
    loc = location[i][location[i].find(char1)+1 : location[i].find(char2)]
    vendor_location.append(loc)
    
    dist = location[i][location[i].find(char2)+1 : location[i].find(char3)]
    dist = dist.replace(',','')
    dist_from_TO.append(int(dist))

In [ ]:
D = {'Year':year,'Make':make,'Model':model,'Price':price,'Mileage':mileage,'Location':vendor_location,'TO_distance':dist_from_TO}

df = pd.DataFrame(D)
df.head()

In [ ]:
df.info()

In [ ]:
df.isna().sum()

In [ ]:
df.dropna(inplace = True)
df['Year'] = df['Year'].astype(int)
# Here dropping all rows with any NaN value in them
# This is not an issue in the grand scheme as there is still a lot of data to complete the analysis with

The mean car of available on this site is described as a 2016 car with almost 62,000 miles prices at $33,000

### More Data Cleaning

In [ ]:
import numpy as np
import pandas as pd
import geopandas
import geopy
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter

In [ ]:
locator = Nominatim(user_agent="myGeocoder")
location = locator.geocode('Port Colborne, Ontario')

In [ ]:
print('Latitude = {}, Longitude = {}'.format(location.latitude, location.longitude))

In [ ]:
df['Location_Region'] = df['Location'] + ', Ontario'

In [ ]:
df.head(2)

In [ ]:
with_loc = df.to_csv('with_loc.csv')

In [ ]:
df = pd.read_csv('with_loc.csv')
df.drop('Location_Region', axis = 1, inplace = True)

### Geocoding the Location of the cars on Sale

In [ ]:
# 1 - convenient function to delay between geocoding calls
geocode = RateLimiter(locator.geocode, min_delay_seconds=1)
# 2- - create location column
df['loc'] = df['Location_Region'].apply(geocode)
# 3 - create longitude, laatitude and altitude from location column (returns tuple)
df['point'] = df['loc'].apply(lambda loc: tuple(loc.point) if loc else None)
# 4 - split point column into latitude, longitude and altitude columns
df[['latitude', 'longitude', 'altitude']] = pd.DataFrame(df['point'].tolist(), index=df.index)

In [ ]:
mapbox_token = 'pk.eyJ1Ijoia2Fuam9tZWxvIiwiYSI6ImNrZWJ6aWM2NzBkcmwzM3FsZXZzcWhxbWMifQ.E-yHj-_8YVnf-KD2-zP0oA'

In [ ]:
df_map = pd.read_csv('with_loc2.csv')
df_map.head(2)

In [ ]:
df_map = df_map[df_map['TO_distance'] < 500]
df_map['Location_Dist'] = df_map['Location'] +', ' + df_map['TO_distance'].astype(str)

In [ ]:
locations = list(df_map['Location'])

In [ ]:
means = df_map.groupby('Location')['Price'].mean()
means = means.to_frame()
means = means.reset_index()
means.head(2)

In [ ]:
df_map = df_map.merge(means, on='Location',how='left')

In [ ]:
Q1 = df_map.quantile(0.25)
Q3 = df_map.quantile(0.75)
IQR = Q3 - Q1
df_map = df_map[~((df_map < (Q1 - 1.5 * IQR)) |(df_map > (Q3 + 1.5 * IQR))).any(axis=1)]

In [ ]:
map_cleaned = df_map.to_csv('map_cleaned.csv')

### Data Visualization

In [ ]:
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
import plotly.graph_objects as go

fig = go.Figure(go.Scattermapbox(
        lat=df_map['latitude'],
        lon=df_map['longitude'],
        mode='markers',
        marker=go.scattermapbox.Marker(
            size=df_map['Price_y']/2500,
            color=df_map['TO_distance'],
            colorscale='electric', 
            showscale=True
            ),
        text=df_map[['Price_y']],
    ))

fig.update_layout(
    autosize=True,
    hovermode='closest',
    mapbox=dict(
        accesstoken=mapbox_token,
        bearing=0,
        center=dict(lat=43.6532,lon=-79.3832),
        pitch=0,
        zoom=6.5
    ),
)

fig.show()

In [ ]:
plt.figure(figsize=(6, 8))
sns.set_style('whitegrid')
bp = sns.boxplot(y=df_map['Price_y'])#, y=df.Price)
bp

### Exploratory Data Analysis

In [ ]:
from scipy import stats
%matplotlib inline

In [ ]:
sns.set_style('whitegrid')
pg = sns.PairGrid(data=df, height = 2.5)
pg.map(plt.scatter, s = 8.0,color = '#538DAA')

There seem to be quite a number of outliers which could heavily skew our analysis, I would like to dive into this further

In [ ]:
plt.figure(figsize=(6, 8))
sns.set_style('whitegrid')
bx = sns.boxplot(y=df['Price'])#, y=df.Price)
bx

In [ ]:
Q1 = df.quantile(0.25)
Q3 = df.quantile(0.75)
IQR = Q3 - Q1
print(IQR)

In [ ]:
df2 = df

In [ ]:
df_out = df2[~((df2 < (Q1 - 1.5 * IQR)) |(df2 > (Q3 + 1.5 * IQR))).any(axis=1)]
df_out.shape

In [ ]:
plt.figure(figsize=(6, 8))
sns.set_style('whitegrid')
bx = sns.boxplot(y=df_out['Price'])#, y=df.Price)
bx

In [ ]:
sns.set_style('whitegrid')
pg = sns.PairGrid(data=df_out, height = 2.5)
pg.map(plt.scatter, s = 8.0,color = '#538DAA')

In [ ]:
sns.heatmap(df_out.corr())

In [ ]:
df_out = df_out.drop('Unnamed: 0', axis =1)

In [ ]:
duplicateDFRow = df_out[df_out.duplicated()]
print(duplicateDFRow)

In [ ]:
df3 = pd.get_dummies(df_out)
df3.head()

In [ ]:
df3.describe()

In [ ]:
sns.set_style('whitegrid')
plt.subplots(figsize=[20,15])
plt.title('Distribution of Car Prices by Car Make', fontsize = 18)
plt.xlabel('Price', fontsize = 18)
plt.ylabel('Make', fontsize = 18)
sns.boxplot(data=df_out, x='Price', y='Make', showfliers=False)

In [ ]:
sns.set_style('whitegrid')
plt.subplots(figsize=[20,15])
plt.title('Distribution of Car Prices by Year', fontsize = 18)
plt.xlabel('Year', fontsize = 18)
plt.ylabel('Price', fontsize = 18)
sns.boxplot(data=df_out, x='Year', y='Price', showfliers=False)

In [ ]:
sns.set_style('whitegrid')
sns.jointplot(data = df_out, x='Mileage', y='Price', color = '#538DAA', kind = 'reg')

In [ ]:
sns.set_style('whitegrid')
sns.jointplot(data = df_out, x='TO_distance', y='Price', color = '#538DAA', kind = 'reg')

#### Training and Testing the Data

In [ ]:
X = df3.drop('Price',axis=1)

y = df3['Price']

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=101)

##### Training the Model

###### Linear Regression

In [ ]:
from sklearn.linear_model import LinearRegression
lr = LinearRegression()

In [ ]:
lr.fit(X_train,y_train)

In [ ]:
# print('Coefficients:\n{0} \n\nIntercept:\n{1}'.format(lr.coef_,lr.intercept_))

In [ ]:
predictions = lr.predict(X_test)

In [ ]:
from sklearn import metrics

In [ ]:
print('MAE:', metrics.mean_absolute_error(y_test, predictions))
print('MSE:', metrics.mean_squared_error(y_test, predictions))
print('RMSE:', np.sqrt(metrics.mean_squared_error(y_test, predictions)))

In [ ]:
sns.set_style('whitegrid')
sns.jointplot(x=y_test, y=predictions, color = '#538DAA', kind = 'reg',
              height = 10)